new model chosen from here: https://huggingface.co/models?sort=downloads&search=bert

In [1]:
import pandas as pd
import transformers

In [2]:
d = pd.read_excel('tweets_labeled.xlsx')
tweets = d['full_text'][:600]
labels = d['Label (critical: 1/neutral: 0)'][:600]
data = pd.DataFrame({'tweets': tweets, 'labels': labels})

this seems to be an improved version of the ct-bert that we took a look at in literature review:
https://huggingface.co/digitalepidemiologylab/covid-twitter-bert-v2-mnli

finetuning is not possible tho.

In [3]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="digitalepidemiologylab/covid-twitter-bert-v2-mnli")

Downloading: 100%|██████████| 833/833 [00:00<00:00, 417kB/s]
Downloading: 100%|██████████| 1.34G/1.34G [01:50<00:00, 12.2MB/s]   
Downloading: 100%|██████████| 364/364 [00:00<00:00, 60.7kB/s]
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 636kB/s] 
Downloading: 100%|██████████| 112/112 [00:00<00:00, 38.0kB/s]


In [5]:
# example from the website
sequence_to_classify = 'To stop the pandemic it is important that everyone turns up for their shots.'
candidate_labels = ['health', 'sport', 'vaccine','guns']
hypothesis_template = 'This example is {}.'
classifier(sequence_to_classify, candidate_labels, hypothesis_template=hypothesis_template, multi_label=True)

{'sequence': 'To stop the pandemic it is important that everyone turns up for their shots.',
 'labels': ['health', 'vaccine', 'sport', 'guns'],
 'scores': [0.9540156126022339,
  0.7107963562011719,
  0.2755419909954071,
  0.0753604918718338]}

In [24]:
sequence_to_classify = "you have to take a shot"
# candidate_labels = ['neutral', 'critical']
candidate_labels = ['critical', 'neutral']
hypothesis_template = 'This example is {}.'
classifier(sequence_to_classify, candidate_labels, hypothesis_template=hypothesis_template, multi_label=True)

{'sequence': 'you have to take a shot',
 'labels': ['critical', 'neutral'],
 'scores': [0.6101869940757751, 0.0019355666590854526]}

In [26]:
for i, t in enumerate(tweets[:50]):
    c = classifier(t, candidate_labels, hypothesis_template=hypothesis_template, multi_label=True)
    label = c['labels'][0]
    if label == 'neutral':
        print(f"{t}\t\t Label: {label}")